In [12]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers,layers

In [4]:

data_dir = os.path.join("../","dataset/data")
image_size = (180, 180)
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=image_size,
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=image_size,
  batch_size=batch_size)

class_names = train_ds.class_names
print("Classes found:", class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 4188 files belonging to 4 classes.
Using 3351 files for training.
Found 4188 files belonging to 4 classes.
Using 837 files for validation.
Classes found: ['Blight', 'Common_Rust', 'Gray_Leaf_Spot', 'Healthy']


In [5]:
print(train_ds)
print(val_ds)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [6]:
IMG_SHAPE = image_size + (3,)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False, 
    weights='imagenet' 
)

base_model.trainable = False


C:\Users\Design Dep\AppData\Local\Temp\ipykernel_7952\448067956.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 10s 1us/step


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=IMG_SHAPE),
    tf.keras.layers.Rescaling(1./127.5, offset=-1),
    base_model, 
    
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding='same',
                  kernel_regularizer=regularizers.l2()),
    layers.Conv2D(filters=64, kernel_size=(3,3), padding='same',
                  kernel_regularizer=regularizers.l2()),
    layers.Activation("relu"),
    layers.Conv2D(filters=64, kernel_size=(3,3), padding='same',
                  kernel_regularizer=regularizers.l2()),
    layers.Activation("relu"),
    layers.MaxPooling2D(pool_size=(2,2), name="pool_1"),
    layers.Dropout(0.3, name="dropout_conv_3"),
    layers.GlobalAveragePooling2D(name="global_avg_pool"),
    layers.Dense(units=256, activation="relu",
                 kernel_regularizer=regularizers.l2()),
    layers.Dropout(0.4, name="dropout_dense_1"),

    layers.Dense(units=128, activation="relu",
                 kernel_regularizer=regularizers.l2()),
    layers.Dropout(0.3),

    layers.Dense(units=4, activation="softmax", name="output_layer")
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])





In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_3 (Rescaling)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 6, 6, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_1_1 (Conv2D)               │ (None, 6, 6, 64)       │       737,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_1_1_1 (Conv2D)             │ (None, 6, 6, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_1_1 (BatchNormalization)     │ (None, 6, 6, 64)       │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_1_2 (Conv2D)               │ (None, 6, 6, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_1_2 (BatchNormalization)     │ (None, 6, 6, 64)       │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_1 (MaxPooling2D)           │ (None, 3, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_conv_1 (Dropout)        │ (None, 3, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2_1 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_2_1 (BatchNormalization)     │ (None, 3, 3, 128)      │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_2_2 (Conv2D)               │ (None, 3, 3, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_2_2 (BatchNormalization)     │ (None, 3, 3, 128)      │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool_2 (MaxPooling2D)           │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_conv_2 (Dropout)        │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_3_1 (Conv2D)               │ (None, 1, 1, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_3_1 (BatchNormalization)     │ (None, 1, 1, 256)      │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_3_2 (Conv2D)               │ (None, 1, 1, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_3_2 (BatchNormalization)     │ (None, 1, 1, 256)      │         1,024 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 4,278,660 (16.32 MB)

 Trainable params: 2,018,884 (7.70 MB)

 Non-trainable params: 2,259,776 (8.62 MB)

In [ ]:
save_path= os.path.join("..", "trained_models/best_model_version.keras")

In [ ]:
model.fit(
    train_ds,
    epochs =20,
    verbose=1,
    validation_data = val_ds,
    callbacks =[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_accuracy",
            patience=5,
            mode="max",
            restore_best_weights=True
        ),
        tf.keras.callbacks.ModelCheckpoint(
            save_path,
            monitor="val_accuracy",
            save_best_only=True,
            verbose=1,
            mode="max"
        )
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.2,         
            patience=5,         
            min_lr=1e-7,        
            verbose=1,
            mode='min'
        )]
)

Epoch 1/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 685ms/step - accuracy: 0.5760 - loss: 6.0634
Epoch 1: val_accuracy improved from -inf to 0.87933, saving model to ..\training_models/best_model_version.keras
105/105 ━━━━━━━━━━━━━━━━━━━━ 104s 907ms/step - accuracy: 0.5775 - loss: 6.0591 - val_accuracy: 0.8793 - val_loss: 4.9926 - learning_rate: 1.0000e-04
Epoch 2/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 566ms/step - accuracy: 0.8845 - loss: 4.9057
Epoch 2: val_accuracy improved from 0.87933 to 0.90442, saving model to ..\training_models/best_model_version.keras
105/105 ━━━━━━━━━━━━━━━━━━━━ 82s 779ms/step - accuracy: 0.8846 - loss: 4.9043 - val_accuracy: 0.9044 - val_loss: 4.4596 - learning_rate: 1.0000e-04
Epoch 3/20
105/105 ━━━━━━━━━━━━━━━━━━━━ 0s 602ms/step - accuracy: 0.9155 - loss: 4.3340
Epoch 3: val_accuracy improved from 0.90442 to 0.91278, saving model to ..\training_models/best_model_version.keras
105/105 ━━━━━━━━━━━━━━━━━━━━ 83s 789ms/step - accuracy: 0.9156 - loss: 4.3328 - val_accuracy: 